In [1]:
import pyodbc
from piLang.piLang.SQLTools import SQLTools
from prettytable import PrettyTable
from piLang.piLang.BoxPlot import BoxPlot
from piLang.piLang.LangValidator import LangValidator, PatternFormat
import time
from validator.BabyDetailValidator import BabyDetailValidator

"""
This code will extract and validate the Babyr detail record in accordance with the QH PDC file format spec.

"""
cnxn = pyodbc.connect('Driver={ODBC Driver 13 for SQL Server}; server=localhost\SQLEXPRESS; database=MaterDB; uid=Shane; password=; trusted_connection=Yes')
cursor = cnxn.cursor()
cursor.execute('SELECT  * FROM dbo.[Babys Detail Record] ORDER BY [Mothers UR number]') 
t = SQLTools(cursor)

cnxn.close()
cnxn = None
#print(t)

baby_detail_meta = { 
    'Transaction Type':                                                 {'Size': 1,  'Type': "char",   'Mandatory': True, 'Enum': ['N','A','D']},
    'Mothers UR number':                                                {'Size': 12, 'Type': "string", 'Mandatory': True, 'Unique': True},
    'Date of confinement':                                              {'Size': 10,  'Type': "date",  'Mandatory': True, 'Format': PatternFormat.DATE_YYMMDD.value},
    'Baby number':                                                      {'Size': 1,  'Type': "int",    'Mandatory': True, 'Expression': '1 <= %1 <=8' },
    'Babys UR number':                                                  {'Size': 12, 'Type': "string", 'Mandatory': True, 'Unique': True},
    'Onset of labour':                                                  {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','3','9']},
    'Induction/ augmentation flag':                                     {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Presentation at birth':                                            {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','4','5','6','7','8','9']},
    'Anaesthesia flag':                                                 {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Method of birth':                                                  {'Size': 2,  'Type': "int",    'Mandatory': True, 'Enum':['10','02','03','04','05','98''99']},
    'Principal accoucheur':                                             {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','3','4','5','6','7','9']},
    'Surgical repair':                                                  {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Labour and deliver complications flag':                            {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Fetal scalp pH':                                                   {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2']},
    'Babies date of birth':                                             {'Size': 10, 'Type': "date",   'Mandatory': True, 'Format': PatternFormat.DATE_YYMMDD.value, 'Expression': 'time.strptime("%1","%d/%m/%Y") == time.strptime("[Date of confinement]","%d/%m/%Y")'},
    'Time of birth':                                                    {'Size': 4,  'Type': "time",   'Mandatory': True, 'Format': PatternFormat.TIME_HHMM.value},
    'Birthweight':                                                      {'Size': 4,  'Type': "int",    'Mandatory': True},
    'Gestation weeks':                                                  {'Size': 2,  'Type': "int",    'Mandatory': True, 'Expression': ''},
    'Plurality':                                                        {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','3','4','5','6','7','8','9']},
    'Babys sex':                                                        {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','3','9']},
    'Born alive/stillborn':                                             {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Macerated':                                                        {'Size': 1,  'Type': "int",    'Mandatory': False, 'Enum':['1','2','9'], 'AllowBlank': True},
    'Vitamin K':                                                        {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','3','9']},
    'Apgar score at 1 minute':                                          {'Size': 2,  'Type': "int",    'Mandatory': True},
    'Apgar score at 5 minutes':                                         {'Size': 2,  'Type': "int",    'Mandatory': True},
    'Regular respirations':                                             {'Size': 2,  'Type': "int",    'Mandatory': False, 'AllowBlank': True},
    'Cord pH':                                                          {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2']},
    'Resuscitation used flag':                                          {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Neonatal morbidity flag':                                          {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Neonatal treatment flag':                                          {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Congenital anomaly flag':                                          {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Puerperium complications flag':                                    {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Separation type - baby':                                           {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','3','4','9']},
    'Baby transferred to':                                              {'Size': 1,  'Type': "int",    'Mandatory': False, 'AllowBlank': True, 'Expression': "if [Separation type - baby] ==2 then %1 !='' else if [Separation type - baby] in (1,3,4,9) then %1 ==''"},
    'Date discharged - baby':                                           {'Size': 10,  'Type': "date",  'Mandatory': False, 'Format': PatternFormat.DATE_YYMMDD.value, 'AllowBlank': True},
    'Intended place of birth':                                          {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','3','4','8','9']},
    'Actual place of birth':                                            {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','3','4','8','9']},
    'Membranes ruptured':                                               {'Size': 5,  'Type': "int",    'Mandatory': True, 'Expression': '"0" <= "%1" <= "99999"'},
    'Length of first stage of labour':                                  {'Size': 5,  'Type': "int",    'Mandatory': False,'Expression': '"0" <= "%1" <= "99999"', 'AllowBlank': True},
    'Length of second stage of labour':                                 {'Size': 5,  'Type': "int",    'Mandatory': False,'Expression': '"0" <= "%1" <= "99999"', 'AllowBlank': True},
    'Reason for forceps/ vacuum':                                       {'Size': 5,  'Type': "string", 'Mandatory': False, 'AllowBlank': True},
    'Cervical dialation prior to caesarean':                            {'Size': 1,  'Type': "int",    'Mandatory': False, 'Enum':['1','2','3'], 'AllowBlank': True},
    'Head circumference at birth':                                      {'Size': 4,  'Type': "float",  'Mandatory': True},
    'Length at birth':                                                  {'Size': 4,  'Type': "float",  'Mandatory': True},
    'Admitted to ICN':                                                  {'Size': 3,  'Type': "int",    'Mandatory': True,'Expression': '"0" <= "%1" <= "999"'},
    'Admitted to SCN':                                                  {'Size': 3,  'Type': "int",    'Mandatory': True,'Expression': '"0" <= "%1" <= "999"'},
    'Reason for admission to ICN/SCN':                                  {'Size': 5,  'Type': "int",    'Mandatory': False, 'AllowBlank': True},
    'Hep B Vaccination':                                                {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'CTG':                                                              {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'FSE':                                                              {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Non-Pharmacological Analgesia flag':                               {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Pharmacological Analgesia flag':                                   {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Fetal scalp pH result':                                            {'Size': 5,  'Type': "float",  'Mandatory': False, 'AllowBlank': True},
    'Cord pH result':                                                   {'Size': 5,  'Type': "float",  'Mandatory': False, 'AllowBlank': True},
    'Water birth flag':                                                 {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Water birth intent':                                               {'Size': 1,  'Type': "int",    'Mandatory': False, 'Enum':['1','2','9'], 'AllowBlank': True},
    'PPH Volume':                                                       {'Size': 1,  'Type': "int",    'Mandatory': False, 'Enum':['1','3','4','9'], 'AllowBlank': True},
    'Fluid(s) the baby recieved in the 24 hours prior to discharge fl': {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Fluid(s) the baby recieved at any time from birth to dicharge fl': {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Extra text flag':                                                  {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2']},
    'Fetal scalp lactate flag':                                         {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2']},
    'Fetal scalp lactate result':                                       {'Size': 4,  'Type': "float",  'Mandatory': False, 'AllowBlank': True},
    'Gestation days':                                                   {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['0','1','2','3','4','5','6','9']},
    'Antibiotics recieved at time of caesarean section':                {'Size': 1,  'Type': "int",    'Mandatory': False, 'Enum':['1','2','3','9'], 'AllowBlank': True},
    'Thromboprophylaxis recieved for caesarean section':                {'Size': 1,  'Type': "int",    'Mandatory': False, 'Enum':['1','2','9'], 'AllowBlank': True},
    'Alternative feeding method flag':                                  {'Size': 1,  'Type': "int",    'Mandatory': False, 'Enum':['1','2','9'], 'AllowBlank': True},
    'Indigenous status (Baby)':                                         {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','3','4','9']},
    'HepatitisBImmunoglobulin':                                         {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2','9']},
    'Perinatal Damage Flag':                                            {'Size': 1,  'Type': "int",    'Mandatory': True, 'Enum':['1','2']},
    'Main Reason for Caesarean':                                        {'Size': 5,  'Type': "string", 'Mandatory': False, 'AllowBlank': True},
    'Main Reason for Caesarean Identifier':                             {'Size': 1,  'Type': "int",    'Mandatory': False, 'Enum':['1','2','3','8'], 'AllowBlank': True},
    'First Additional Reason for Caesarean':                            {'Size': 5,  'Type': "string", 'Mandatory': False, 'AllowBlank': True},
    'First Additional Reason for Caesarean Identifier':                 {'Size': 1,  'Type': "int",    'Mandatory': False, 'Enum':['1','2','3','8'], 'AllowBlank': True},
    'Second Additional Reason for Caesarean':                           {'Size': 5,  'Type': "string", 'Mandatory': False, 'AllowBlank': True},
    'Second Additional Reason for Caesarean Identifier':                {'Size': 1,  'Type': "int",    'Mandatory': False, 'Enum':['1','2','3','8'], 'AllowBlank': True},
    'Main Reason for induction':                                        {'Size': 5,  'Type': "string", 'Mandatory': False, 'AllowBlank': True},
    'Reason for induction Additional 1':                                {'Size': 5,  'Type': "string", 'Mandatory': False, 'AllowBlank': True},
    'Reason for induction Additional 2':                                {'Size': 5,  'Type': "string", 'Mandatory': False, 'AllowBlank': True}
}
        
print("\n---> Running langvalidator ...")
stime = time.time()
lang_validator = LangValidator(t.rs, baby_detail_meta)
lang_validator.validate()
lang_validator.counters.calcMean()
lang_validator.counters.calcPercent()
print("Completed in " + str(time.time() - stime) + " secs")
print(lang_validator.getErrors())
print(lang_validator.counters.toList())

print("\n--> Plot:")

x = BoxPlot()
x.plot(lang_validator.counters.toList())

print("\n\n### DONE ###")



---> Running langvalidator ...


Completed in 9.02286148071289 secs
Error: Field 'Mothers UR number' with value '6302756.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6302756.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6305900.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6305900.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6314740.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6314740.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6317863.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6317863.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6326215.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6326215.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6327716.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6327716.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6328358.0' is not UNIQUE
Error: Field 'Mothers UR number' with value '6328358.0'



### DONE ###
